# S-P visualization using data from SC3 scautopick


1. Use SC3 to pick P and S using scautopick; this creates sc3ml.
2. Parse sc3ml to produce a text file.
3. This notebook provides a basic visualization of the text file with S-P values. Also included in the file is the S-P SNR value.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys

%matplotlib inline

In [ ]:
names = ['dt', 'sp', 'snr']

site = 'WIZ'

date1 = '20190527'
date2 = '20190531'

#construct and format the range of dates
dr = pd.date_range(date1, date2, freq='D', )
dates = dr.map(lambda x: x.strftime('%Y%m%d'))

## Load and prepare data

In [ ]:
df  = pd.DataFrame()

for date in dates:
    xmlfile = site+'_'+date+'_sc3_s-p.dat'
    print (xmlfile)
    dfile = pd.read_csv(xmlfile, delim_whitespace=True, header=None, parse_dates=[0])
    df = df.append(dfile)

In [ ]:
df.columns = names
df.set_index('dt', inplace=True)

#replace snr values above some threshold
df['snr'].values[df['snr'].values > 15] = 15

In [ ]:
#data set with SNR >=10
dfbest = df[df['snr']>=10]

## Visualize

### Time-series

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1,1,1)

plt.scatter(df.index, df.sp, c=df.snr, edgecolors='None', s=20, cmap='binary')
plt.colorbar(label='S-P SNR')

plt.ylim(0,4)
plt.xlim(date1, date2)
plt.ylabel('S-P (sec)')

plt.title('S-P at '+site)

plt.savefig(site+'_S-P_time-series.png')

### Histogram

In [ ]:
#bins, full range of observations, 0.1 second bins
bins = np.arange(np.around(df['sp'].values.min(), decimals=1), df['sp'].values.max(), 0.1)

title = 'Distribution of S-P at '+ site
fig,(ax) = plt.subplots(1, 1, figsize=(8,6))
df['sp'].plot.hist(ax=ax, bins=bins, label='all data', title=title)
dfbest['sp'].plot.hist(ax=ax, bins=bins, label='best data')
ax.legend(loc='best')

plt.savefig(site+'_S-P_histogram.png')